In [1]:
import sys
sys.path.insert(1, "../")
sys.path.insert(1, "../Models/")
import torch
from torch.utils.data import Subset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from detection_datasets import *
from yolov2 import YOLOv2D19 as YOLOv2
from data_preprocessing import get_norms
from albumentations.pytorch import ToTensorV2
import albumentations as A
from eval import *

C:\Vstanovlene\Anaconda Distribution\envs\new_gpu_env\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
import gc

# Invoke garbage collector
gc.collect()

# Clear GPU cache
torch.cuda.empty_cache()
print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

Allocated memory: 0.00 MB
Allocated memory: 0.00 MB


In [3]:
import pickle
with open('../Models/anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

In [4]:
device = torch.device('cuda:0')
# device = torch.device('cpu')
dtype=torch.float32

In [5]:
norms = get_norms('../../datasets/VOCdevkit/trainval_norms.json')
means = norms['means']
stds = norms['stds']

In [6]:
model = YOLOv2(state_dict_path='./darknet19_72.96.pth', device=device, dtype=dtype, num_anchors=5)

C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Models\../Models\yolov2.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location=

In [7]:
transforms = A.Compose([
    A.Resize(width=416, height=416),
    # A.VerticalFlip(p=1.0),
    A.Normalize(mean=means, std=stds),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', 
                         subsets = [('VOC2012', 'trainval')],
                         scales=[13], anchors=anchors, transforms=transforms, 
                         dtype=dtype, device=device)
val_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', 
                       subsets = [('VOC2007', 'val')],
                       scales=[13], anchors=anchors, transforms=transforms, 
                       dtype=dtype, device=device)

True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\val.txt


In [8]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False)

In [9]:
model = YOLOv2(state_dict_path='./darknet19_72.96.pth', device=device, dtype=dtype, num_anchors=5)

In [10]:
evaluate(model, loader=val_loader, anchors=anchors)

batch 0 2025-03-15 23:25:58.831834


C:\Vstanovlene\Anaconda Distribution\envs\new_gpu_env\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


batch 1 2025-03-15 23:26:00.634435
batch 2 2025-03-15 23:26:02.169646
batch 3 2025-03-15 23:26:03.569033
batch 4 2025-03-15 23:26:05.144547
batch 5 2025-03-15 23:26:06.584536
batch 6 2025-03-15 23:26:08.091271
batch 7 2025-03-15 23:26:09.636529
batch 8 2025-03-15 23:26:11.230546
batch 9 2025-03-15 23:26:12.661851
batch 10 2025-03-15 23:26:14.137801
batch 11 2025-03-15 23:26:16.036941
batch 12 2025-03-15 23:26:17.576725
batch 13 2025-03-15 23:26:19.002605
batch 14 2025-03-15 23:26:20.546336
batch 15 2025-03-15 23:26:22.006161
batch 16 2025-03-15 23:26:23.342437
batch 17 2025-03-15 23:26:24.881977
batch 18 2025-03-15 23:26:26.514172
batch 19 2025-03-15 23:26:27.999413
batch 20 2025-03-15 23:26:29.652513
batch 21 2025-03-15 23:26:31.065484
batch 22 2025-03-15 23:26:32.481983
batch 23 2025-03-15 23:26:34.052163
batch 24 2025-03-15 23:26:35.476258
batch 25 2025-03-15 23:26:37.064890
batch 26 2025-03-15 23:26:38.502722
batch 27 2025-03-15 23:26:40.091970
batch 28 2025-03-15 23:26:41.678957
b

KeyboardInterrupt: 

In [30]:
from torchmetrics.detection import MeanAveragePrecision

def evaluate(model, loader, mAP=MeanAveragePrecision(box_format='cxcywh',class_metrics=True),
            num_classes=20, num_boxes=5, obj_threshold=0.5, iou_threshold=0.5):
    for i, (imgs, labels) in enumerate(loader):
        print(f'batch {i} {datetime.datetime.now()}')
        preds = []
        targets = []
        batch_size = labels.shape[0]
        with torch.no_grad():
            out = model(imgs).detach()
            pred_boxes = get_pred_boxes(out, anchors, num_classes=num_classes, num_boxes=num_boxes, obj_threshold=obj_threshold)
            gt_boxes = get_gt_boxes(labels.detach(), num_classes=num_classes, num_boxes=num_boxes, obj_threshold=obj_threshold)
            keep = NMS(pred_boxes, num_classes=num_classes, iou_threshold=iou_threshold)
            for j in range(batch_size):
                boxes = pred_boxes[j][keep[j], ..., 1:5]
                scores = pred_boxes[j][keep[j], ..., 0]
                labels = pred_boxes[j][keep[j], ..., 5].to(torch.uint8)
                preds.append({'boxes':boxes,
                             'scores':scores,
                             'labels':labels})
    
                boxes = gt_boxes[j][..., 1:5]
                labels = gt_boxes[j][..., 5].to(torch.uint8)
                targets.append({'boxes':boxes,
                               'labels':labels})
    
        mAP.update(preds=preds, target=targets)
    result = mAP.compute()
    mAP.reset()
    return result